In [1]:
import numpy as np
import sys

In [2]:
#Getting test data
#data.py
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
def get_data():
    #Put custom data retrieve function here!
    labels = ["T-shirt/top","Pantalone","Pullover","Vestito","Cappotto","Sandalo","Maglietta","Sneaker","Borsa","Stivaletto"]
    num_classes=10
    ((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
    #reshape
    trainX = trainX.reshape(trainX.shape[0],28*28) 
    testX = testX.reshape(testX.shape[0],28*28)
    #scale data to range [0,1]
    trainX=trainX/255
    testX=testX/255
    #one-shot encode the training and testing label
    trainY = to_categorical(trainY, num_classes)
    testY = to_categorical(testY, num_classes)
    return trainX, trainY, testX, testY



Using TensorFlow backend.
c:\users\lafacero\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\lafacero\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\lafacero\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [9]:
#model.py
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation 
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense


def model(opt):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim=784))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu')) 
    model.add(Dense(10, activation='softmax'))
    model.summary()

    #Compile
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model


In [10]:
#train.py
import os
from keras import backend as K
from keras.callbacks import History 
import keras
import tensorflow as tf
import numpy as np
import sys
sys.path.append('model/')
sys.path.append('data/')
#from model import model
import json
import mlflow

def train(trainX, trainY, model):
    history = History()
    print('Loading params from json..')
    with open('training_conf.json') as f:
        data = json.load(f)
        #print('Current data params are: ')
        #print(data)
    
    model.fit(trainX, trainY, batch_size=data['batch_size'], epochs=data['epochs'], verbose=data['verbose'], validation_split=0.3, shuffle=data['shuffle'], callbacks=[history])
    return model


In [11]:
#main 
#Thanks to Analytic Vidhya
#See: https://medium.com/analytics-vidhya/tracking-deep-learning-experiments-using-keras-mlflow-and-mongodb-732fc771266c
import json
import mlflow
import sys
sys.path.append('model/')
sys.path.append('data/')
#from train import train
#from data import get_data

def main():
    with open('training_conf.json') as f:
        data = json.load(f)
        print(data)
    
    trainX, trainY, testX, testY = get_data()
    keras_model = model(data['opt'])
    
    with mlflow.start_run(run_name=data['name']):    
        trained_model=train(trainX, trainY, keras_model)
        scores=keras_model.evaluate(testX, testY,verbose=1)
        mlflow.log_param("alpha", 0.001)
        mlflow.log_param("epochs", data['epochs'])
        mlflow.log_param("optimizer", data['opt'])
        mlflow.log_param("batch_size", data['batch_size'])
        mlflow.log_param("eval_loss", scores[0])
        mlflow.log_param("val_acc", scores[1])
        #mlflow.log_param("eval_precision", scores[2])
        #mlflow.log_param("eval_recall", scores[3])
        #mlflow.log_param(key="accuracy", value=scores[4], step=dataset_count)

if __name__ == "__main__":
    main()        

{'name': 'fashion_mnist', 'batch_size': 512, 'epochs': 10, 'verbose': 'True', 'shuffle': 'True', 'opt': 'sgd'}


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 567,434
Trainable params: 567,434
Non-trainable params: 0
_________________________________________________________________


Loading params from json..
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 42000 samples, validat

In [ ]:
!mlflow ui